In [ ]:
!pip install gradio


In [ ]:
# Import required libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import joblib
import numpy as np
import gradio as gr

# Step 1: Load the datasets
file_path = "/content/drive/MyDrive/datasets for pnd/Final_Cleaned_Health_Nutrition_Dataset.csv"
health_data = pd.read_csv(file_path)

# Step 2: Data Preprocessing
def preprocess_user_data(user_data):
    """Add BMI as a feature and encode Activity Level."""
    user_data['BMI'] = user_data['Weight (kg)'] / (user_data['Height (cm)'] / 100) ** 2
    activity_map = {'Low': 1, 'Medium': 2, 'High': 3}
    user_data['Activity_Level'] = user_data['Weight_Status'].str.strip().str.capitalize()
    user_data['Activity_Level'] = user_data['Activity_Level'].map(activity_map).fillna(1)
    return user_data

health_data = preprocess_user_data(health_data)

# Step 3: Train Calorie Prediction Model
def train_calorie_model(user_data):
    """Train a calorie prediction model."""
    X = user_data[['Age', 'Weight (kg)', 'Height (cm)', 'Activity_Level']].dropna()
    y = user_data['Caloric_Intake (kcal)'].dropna()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    print("Calorie Prediction Model Performance:")
    print("MSE:", mean_squared_error(y_test, model.predict(X_test)))
    joblib.dump(model, "calorie_predictor.pkl")
    return model

calorie_model = train_calorie_model(health_data)

# Step 4: Food Recommendation System
def recommend_food(target_calories, dietary_preference, food_data, meal_type):
    """Recommend food based on target calories and dietary preference."""
    filtered_food = food_data[
        (food_data['Diet_Preference'].str.lower() == dietary_preference.lower()) &
        (food_data['Weight_Goal'].str.lower() == meal_type.lower())
    ]
    if filtered_food.empty:
        return food_data.sample(n=3)
    recommendations = filtered_food[filtered_food['Caloric_Intake (kcal)'] <= target_calories / 3]
    return recommendations.sample(n=3) if not recommendations.empty else filtered_food.sample(n=3)

# Step 5: Exercise Recommendation System
def recommend_exercises(activity_level, exercise_data, disease=None):
    """Recommend exercises based on activity level and health conditions."""
    if disease:
        if 'Existing_Condition' in exercise_data.columns:
            filtered_exercises = exercise_data[
                exercise_data['Existing_Condition'].str.contains(disease, case=False, na=False)
            ]
        else:
            filtered_exercises = exercise_data  # Fallback to all exercises if column is missing
    else:
        filtered_exercises = exercise_data[exercise_data['Activity_Level'] == activity_level]
    return filtered_exercises.sample(n=3) if not filtered_exercises.empty else exercise_data.sample(n=3)

# Step 6: Gradio Interface
def nutritional_advisor(name, age, weight, height, gender, goal, dietary_preference, activity_level, disease):
    calorie_model = joblib.load("calorie_predictor.pkl")
    activity_map = {'low': 1, 'medium': 2, 'high': 3}
    activity_level_encoded = activity_map.get(activity_level.lower(), 1)
    user_features = np.array([[age, weight, height, activity_level_encoded]])
    bmr = calorie_model.predict(user_features)[0]

    # Calculate BMI
    bmi = weight / (height / 100) ** 2

    # Adjust target calories based on goal
    if goal.lower() == 'weight loss':
        target_calories = bmr - 500
    elif goal.lower() == 'weight gain':
        target_calories = bmr + 500
    else:
        target_calories = bmr

    # Generate recommendations
    diet_plan = {
        "Breakfast": recommend_food(target_calories, dietary_preference, health_data, "breakfast"),
        "Lunch": recommend_food(target_calories, dietary_preference, health_data, "lunch"),
        "Dinner": recommend_food(target_calories, dietary_preference, health_data, "dinner")
    }
    workout_plan = recommend_exercises(activity_level_encoded, health_data, disease)

    # Format output
    output = f"Hello {name}, here are your personalized recommendations:\n\n"
    output += f"Your BMI: {bmi:.2f}\n\n"
    output += "Diet Plan:\n"
    for meal, items in diet_plan.items():
        output += f"{meal}:\n{items[['Recommended_Foods_Clean', 'Caloric_Intake (kcal)', 'Nutritional_Info']].to_string(index=False)}\n\n"
    output += "Workout Plan:\n"
    output += workout_plan[['Recommended_Exercises']].to_string(index=False)
    output += "\n\nGeneral Instructions:\n"
    if goal.lower() == 'weight loss':
        output += "Focus on a calorie deficit. Incorporate cardio and strength training.\n"
    elif goal.lower() == 'weight gain':
        output += "Focus on a calorie surplus. Include weight training and protein-rich foods.\n"
    else:
        output += "Maintain your calorie intake and continue a balanced diet and exercise.\n"
    return output

# Extract unique diseases from the dataset
if 'Existing_Condition' in health_data.columns:
    unique_diseases = health_data['Existing_Condition'].dropna().str.split(',').explode().str.strip().unique().tolist()
else:
    unique_diseases = []

# Define Gradio interface
interface = gr.Interface(
    fn=nutritional_advisor,
    inputs=[
        gr.Textbox(label="Name"),
        gr.Number(label="Age"),
        gr.Number(label="Weight (kg)"),
        gr.Number(label="Height (cm)"),
        gr.Radio(["Male", "Female"], label="Gender"),
        gr.Radio(["Weight Loss", "Weight Gain", "Maintenance"], label="Goal"),
        gr.Radio(["Vegan", "Vegetarian", "Non-Vegetarian"], label="Dietary Preference"),
        gr.Radio(["Low", "Medium", "High"], label="Activity Level"),
        gr.Dropdown(choices=unique_diseases, label="Health Condition", multiselect=False),
    ],
    outputs="text",
    title="Nutritional Advisor",
    description="Personalized dietary and workout recommendations based on your profile and preferences.",
    css="""
    body {
        background-color: #001f3f;
        color: #f5f5f5;
    }
    .input, .output {
        color: #f5f5f5;
    }
    """
)

# Launch Gradio interface
interface.launch()


Calorie Prediction Model Performance:
MSE: 7477.922982161353
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://992634ff5031ffeb28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
